In [2]:

lat = 40.6851482
lon = -73.9255693


select = '&$select=date_extract_y(crash_date) as year,SUM(number_of_persons_injured) as injured, SUM(number_of_persons_killed) as killed, SUM(number_of_pedestrians_injured) as ped_injured, SUM(number_of_pedestrians_killed) as ped_killed, SUM(number_of_cyclist_injured) as cyclist_injured, SUM(number_of_cyclist_killed) as cyclist_killed, SUM(number_of_motorist_injured) as motorist_injured, SUM(number_of_motorist_killed) as motorist_killed'
groupby = '&$group=year'
collusions = f'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$where=within_circle(location,{lat},{lon},1000){select}{groupby}'

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

# Get the data
r = requests.get(collusions)
data = r.json()

# Convert to dataframe
df = pd.DataFrame(data)
print(df.shape)
display(df.head())

df = df.astype(int)

# plotly area chart
import plotly.express as px
fig = px.area(df, x="year", y=["motorist_killed", "ped_killed", "cyclist_killed","motorist_injured", "ped_injured", "cyclist_injured"], title='Injuries and Deaths')

# hovermode = 'x unified' to compare across years
fig.update_layout(hovermode = 'x unified')

# hovertemplate showing percentages not years
fig.update_traces(hovertemplate = '%{y}')


fig.show()



(12, 9)


,year,injured,killed,ped_injured,ped_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed
0,2012,236,1,56,0,18,0,162,1
1,2013,470,0,96,0,39,0,335,0
2,2014,451,0,100,0,48,0,303,0
3,2015,405,3,68,2,40,0,297,1
4,2016,442,1,80,1,45,0,316,0


In [1]:
# Bed-Stuy: 40.6871213406883, -73.92360017855678
# Greenpoint: 40.7334195, -73.957095
# Hancock: 40.6851482 -73.9255693
lat = 40.6851482
lon = -73.9255693


where = f'$where=within_circle(Lon_Lat,{lat},{lon},1000) AND year > 2016 AND ofns_desc NOT IN ("ADMINISTRATIVE CODE","OFFENSES AGAINST PUBLIC ADMINISTRATION","FRAUDS","OTHER STATE LAWS (NON PENAL LA","GAMBLING", "FOR OTHER AUTHORITIES","NEW YORK CITY HEALTH CODE", "CHILD ABANDONMENT/NON SUPPORT 1", "UNAUTHORIZED USE OF A VEHICLE 3 (UUV)","", "FORGERY", "MISCELLANEOUS PENAL LAW","OTHER STATE LAWS (NON PENAL LAW)")'
select = f'&$select= ofns_desc, COUNT(ofns_desc) , date_extract_y(arrest_date) as year'
groupby = '&$group=year, ofns_desc'
arr_url = f'https://data.cityofnewyork.us/resource/8h9b-rp9u.json?{where}{select}{groupby}'

r = requests.get(arr_url)
data = r.json()

df = pd.DataFrame(data)
df['COUNT_ofns_desc'] = df['COUNT_ofns_desc'].astype(int)
df['year'] = df['year'].astype(int)

df['ofns_desc'] = df['ofns_desc'].str.replace(' 2', '')
df['ofns_desc'] = df['ofns_desc'].str.replace(' 5', '')

sort = df.groupby('ofns_desc').sum().sort_values('COUNT_ofns_desc', ascending=False).index
df['ofns_desc'] = pd.Categorical(df['ofns_desc'], categories=sort, ordered=True)

import plotly.express as px
# make stacked area chart
fig = px.area(df, x="year", y="COUNT_ofns_desc", color="ofns_desc", line_group="ofns_desc", category_orders={'ofns_desc':sort}, title='Arrests')
# sort color legend by count
fig.update_traces(hovertemplate="Year: %{x}<br>Count: %{y}")

# remove half x-tick labels
fig.update_xaxes( tickvals= df['year'].unique() )

fig.show()


NameError: name 'requests' is not defined

In [64]:
sort = df.groupby('ofns_desc').sum().sort_values('COUNT_ofns_desc', ascending=False).index

Index(['ASSAULT 3 & RELATED OFFENSES', 'DANGEROUS DRUGS', 'FELONY ASSAULT',
       'ROBBERY', 'DANGEROUS WEAPONS', 'OTHER OFFENSES RELATED TO THEFT',
       'PETIT LARCENY', 'VEHICLE AND TRAFFIC LAWS', 'BURGLARY',
       'CRIMINAL MISCHIEF & RELATED OF', 'GRAND LARCENY',
       'CRIMINAL MISCHIEF & RELATED OFFENSES',
       'PROSTITUTION & RELATED OFFENSES', 'CRIMINAL TRESPASS',
       'OTHER TRAFFIC INFRACTION', 'POSSESSION OF STOLEN PROPERTY',
       'INTOXICATED & IMPAIRED DRIVING', 'OFF. AGNST PUB ORD SENSBLTY &',
       'OFFENSES AGAINST PUBLIC ADMINI', 'OTHER OFFENSES RELATED TO THEF',
       'OFFENSES INVOLVING FRAUD', 'GRAND LARCENY OF MOTOR VEHICLE',
       'SEX CRIMES', 'OTHER STATE LAWS', 'MURDER & NON-NEGL. MANSLAUGHTE',
       'OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV',
       'MURDER & NON-NEGL. MANSLAUGHTER', 'OFFENSES AGAINST THE PERSON',
       'INTOXICATED/IMPAIRED DRIVING', 'UNAUTHORIZED USE OF A VEHICLE',
       'NYS LAWS-UNCLASSIFIED FELONY', 'F.C.A. P.I.N.O.S.'